<a href="https://colab.research.google.com/github/Sam-uel-Codes/AI/blob/main/5_flowers_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> 5 Flower Classification </h1>


<h2> Importing Libraries and Modules </h2>

In [ ]:
# !pip install wandb

# import wandb
# from wandb.keras import WandbCallback

# import tensorflow as tf
# import matplotlib.pyplot as plt
# from tensorflow import keras

In [ ]:
# Install Weights & Biases if not already installed
!pip install --upgrade wandb

import wandb
from wandb.integration.keras import WandbMetricsLogger
import tensorflow as tf
from tensorflow import keras

# Constants
IMG_CHANNELS = 3
CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

def read_and_decode(filename, resize_dims):
    img_bytes = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, resize_dims)
    return img

def parse_csvline(csv_line, img_height, img_width):
    record_default = ["", ""]
    filename, label_string = tf.io.decode_csv(csv_line, record_default)
    img = read_and_decode(filename, [img_height, img_width])
    label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
    return img, label

def train():
    # Initialize a W&B run for this training instance
    wandb.init(project="flowers-classifier-sweep")
    config = wandb.config

    IMG_HEIGHT = config.img_height
    IMG_WIDTH = config.img_width
    BATCH_SIZE = config.batch_size
    EPOCHS = config.epochs
    LEARNING_RATE = config.learning_rate
    HIDDEN_UNITS = config.hidden_units

    # Prepare datasets with dynamic image dimensions and batch size
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(lambda line: parse_csvline(line, IMG_HEIGHT, IMG_WIDTH), num_parallel_calls=tf.data.AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(lambda line: parse_csvline(line, IMG_HEIGHT, IMG_WIDTH), num_parallel_calls=tf.data.AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Define the model
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(HIDDEN_UNITS, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=["accuracy"]
    )

    # Train the model with W&B logging enabled
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=EPOCHS,
        callbacks=[WandbMetricsLogger()]
    )
    wandb.finish()

# Sweep configuration
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {'values': [5, 10]},
        'batch_size': {'values': [8, 16]},
        'img_height': {'values': [16, 32]},
        'img_width': {'values': [16, 32]},
        'learning_rate': {'values': [0.001, 0.0001]},
        'hidden_units': {'values': [64, 128]}
    }
}

# Create and launch the sweep
sweep_id = wandb.sweep(sweep_config, project="5-flowers-classification")
wandb.agent(sweep_id, function=train)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: s4oxt31d
Sweep URL: https://wandb.ai/samtrieswnb-vellore-institute-of-technology/5-flowers-classification/sweeps/s4oxt31d


...

Epoch 1/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 132s 316ms/step - accuracy: 0.3366 - loss: 1.5041 - val_accuracy: 0.4486 - val_loss: 1.2470
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 137s 306ms/step - accuracy: 0.4563 - loss: 1.2577 - val_accuracy: 0.4811 - val_loss: 1.2128
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 121s 293ms/step - accuracy: 0.4954 - loss: 1.2015 - val_accuracy: 0.4919 - val_loss: 1.2212
Epoch 4/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 203s 440ms/step - accuracy: 0.5132 - loss: 1.1560 - val_accuracy: 0.4892 - val_loss: 1.2196
Epoch 5/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 150s 313ms/step - accuracy: 0.5414 - loss: 1.1168 - val_accuracy: 0.4757 - val_loss: 1.2238


epoch/accuracy,▁▄▆▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▄▃▂▁
epoch/val_accuracy,▁▆██▅
epoch/val_loss,█▁▃▂▃
epoch/accuracy,0.54121
epoch/epoch,4
epoch/learning_rate,0.001
epoch/loss,1.1208
epoch/val_accuracy,0.47568


wandb: Agent Starting Run: eejd7b4r with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_units: 64
wandb: 	img_height: 16
wandb: 	img_width: 32
wandb: 	learning_rate: 0.0001


Epoch 1/5
     22/Unknown 6s 240ms/step - accuracy: 0.1934 - loss: 1.6797

wandb: Ctrl + C detected. Stopping sweep.
